In [57]:
import pandas as pd

# Read the CSV file with semicolon as the delimiter
df = pd.read_csv("/Users/ulrike_imac_air/projects/DataScienceProjects/forbes_project/forbes_csv/forbes_gps.csv", sep=';')

# Display the first few rows to verify that the file is read correctly
display(df.head())
display(df.info(verbose=True))

,ID_location,City,US_State,Country,Continent,Unnamed: 5
0,1,A Coruña,NaN,Spain,South America,NaN
1,2,Aarhus,NaN,Denmark,Europe,NaN
2,3,Aberdeen,NaN,UK,Europe,NaN
3,4,Aberdyfi,NaN,UK,Europe,NaN
4,5,Abu Dhabi,NaN,United Arab Emirates,Asia,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 693 entries, 0 to 692
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ID_location  693 non-null    int64  
 1   City         693 non-null    object 
 2   US_State     256 non-null    object 
 3   Country      693 non-null    object 
 4   Continent    693 non-null    object 
 5   Unnamed: 5   0 non-null      float64
dtypes: float64(1), int64(1), object(4)
memory usage: 32.6+ KB


None

As a next step, let’s figure out what kind of unique Continents and Country_of_residence we have:

In [58]:
""" unique_continents = df["Continent"].unique()

for item in unique_continents:
    print(unique_continents) """

' unique_continents = df["Continent"].unique()\n\nfor item in unique_continents:\n    print(unique_continents) '

In [60]:
gr_cat = df[["Continent",
             "Country"]].groupby(["Continent",
                                       "Country"], as_index=False).size()

In [61]:
import plotly.express as px

fig = px.sunburst(gr_cat, width=1280, height=800,
                  path=["Continent", "Country"], values="size",
                  color="Continent",
                  title="<span style='font-size:18px;'><b>Forbes billionaires - going by continents and countries (cities in total number)</b></span><b></b>"
                  )
fig.update_layout(font_size=10, margin=dict(l=10, r=10, t=30, b=50))
fig.update_traces(textinfo="label+percent parent")
fig.show()

In [62]:
gr_city = df[["Continent",
             "Country", "City"]].groupby(["Continent",
                                       "Country", "City"], as_index=False).size()

In [63]:
import plotly.express as px

fig = px.sunburst(gr_city, width=1280, height=800,
                  path=["Continent", "Country", "City"], values="size",
                  color="Continent",
                  title="<span style='font-size:18px;'><b>Forbes billionaires - going by continents, countries, and cities (in total number)</b></span><b></b>"
                  )
fig.update_layout(font_size=10, margin=dict(l=10, r=10, t=30, b=50))
fig.update_traces(textinfo="label+percent parent")
fig.show()


In [64]:
fig = px.treemap(gr_cat, width=1280, height=800,
                 path=['Continent', 'Country'], values='size',
                 color='Continent')
fig.update_traces(textinfo="label+percent parent")
fig.show()

In [65]:
gr_us_state = df[["US_State",
             "City"]].groupby(["US_State",
                                "City"], as_index=False).size()

In [66]:
import plotly.express as px

fig = px.sunburst(gr_us_state, width=1280, height=800,
                  path=["US_State", "City"], values="size",
                  color="US_State",
                  title="<span style='font-size:18px;'><b>Forbes billionaires - going by US States (cities in total number)</b></span><b></b>"
                  )
fig.update_layout(font_size=10, margin=dict(l=10, r=10, t=30, b=50))
fig.update_traces(textinfo="label+percent parent")
fig.show()

Locations
Let's try and get the GPS data

In [67]:
gr_location = df[["Country"]].groupby(['Country'], as_index=False).size().sort_values(by="size", ascending=False)
display(gr_location[:10])

,Country,size
68,USA,256
14,China,102
25,Germany,49
61,Switzerland,32
29,India,22
67,UK,20
33,Italy,19
8,Brazil,15
54,Russia,13
11,Canada,12


I will use a Python geopy library to get the coordinates:

In [73]:
from geopy.geocoders import Nominatim
from typing import Tuple
from functools import lru_cache

geolocator = Nominatim(user_agent="Python3.12")

@lru_cache(maxsize=None)
def get_coord_lat_lon(city_name: str, country_name: str = None) -> Tuple[float, float]:
    """ Get coordinates for Cities """
    if country_name:
        location = geolocator.geocode(city_name + ', ' + country_name)
    else:
        location = geolocator.geocode(city_name)
    return (location.latitude, location.longitude) if location else (None, None)

# Create empty lists to store latitude and longitude
latitudes = []
longitudes = []

# Iterate over each row in the DataFrame
for index, row in df.iterrows():
    city_name = row['City']
    country_name = row['Country'] 

    lat, lon = get_coord_lat_lon(city_name, country_name)
    if lat is not None and lon is not None:
        latitudes.append(lat)
        longitudes.append(lon)
    else:
        latitudes.append(None)
        longitudes.append(None)

# Add latitude and longitude columns to the DataFrame
df['latitude'] = latitudes
df['longitude'] = longitudes

# Display the DataFrame with latitude and longitude columns
display(df.head())


,ID_location,City,US_State,Country,Continent,Unnamed: 5,latitude,longitude
0,1,A Coruña,NaN,Spain,South America,NaN,43.370970,-8.395943
1,2,Aarhus,NaN,Denmark,Europe,NaN,56.149628,10.213405
2,3,Aberdeen,NaN,UK,Europe,NaN,57.148243,-2.092809
3,4,Aberdyfi,NaN,UK,Europe,NaN,52.545125,-4.053362
4,5,Abu Dhabi,NaN,United Arab Emirates,Asia,NaN,24.453835,54.377401


In [74]:
df.to_csv('data_with_coordinates_new.csv', index=False)

In [75]:
# Assuming 'df' is your DataFrame containing latitude and longitude columns
df.to_csv('/Users/ulrike_imac_air/projects/DataScienceProjects/forbes_project/forbes_csv/data_with_coordinates_new2.csv', index=False)

display(df.head())
display(df.info(verbose=True))

,ID_location,City,US_State,Country,Continent,Unnamed: 5,latitude,longitude
0,1,A Coruña,NaN,Spain,South America,NaN,43.370970,-8.395943
1,2,Aarhus,NaN,Denmark,Europe,NaN,56.149628,10.213405
2,3,Aberdeen,NaN,UK,Europe,NaN,57.148243,-2.092809
3,4,Aberdyfi,NaN,UK,Europe,NaN,52.545125,-4.053362
4,5,Abu Dhabi,NaN,United Arab Emirates,Asia,NaN,24.453835,54.377401


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 693 entries, 0 to 692
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ID_location  693 non-null    int64  
 1   City         693 non-null    object 
 2   US_State     256 non-null    object 
 3   Country      693 non-null    object 
 4   Continent    693 non-null    object 
 5   Unnamed: 5   0 non-null      float64
 6   latitude     689 non-null    float64
 7   longitude    689 non-null    float64
dtypes: float64(3), int64(1), object(4)
memory usage: 43.4+ KB


None

Here, I used the lru_cache decorator, which may be helpful if I want to run the code several times; the data will be taken from the cache instead of a new API call. I also used a tqdm Python library that allows me to see a progress bar during the processing—it’s useful because the process takes several minutes:

In [76]:
gr_city_location = df[["City"]].groupby(['City'], as_index=False).size().sort_values(by="size", ascending=False)
display(gr_city_location[:10])

,City,size
520,Saint Petersburg,2
630,Vienna,2
337,Lexington,2
0,A Coruña,1
462,Panzhihua,1
455,Oviedo,1
456,Oyster Bay,1
457,Pacific Palisades,1
458,Palisades,1
459,Palm Beach,1


In [77]:
import folium
from branca.element import Figure

# Create a Figure object
fig = Figure(width=1024, height=600)

# Create a Folium Map object covering the whole world
fmap = folium.Map(location=[0, 0], tiles="openstreetmap", zoom_start=2)

# Filter out rows with NaN latitude or longitude
df_filtered = df.dropna(subset=['latitude', 'longitude'])

# Iterate over each row in the filtered DataFrame
for index, row in df_filtered.iterrows():
    latitude, longitude = row['latitude'], row['longitude']
    name = row["City"] 
    # Add marker to the map
    folium.Marker(location=[latitude, longitude], popup=name).add_to(fmap)

# Add the Folium Map object to the Figure
fig.add_child(fmap)

# Display the Figure
display(fig)
